In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import image_loader
import torch
from torchvision import transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt 
csv_file = '../data/movie_data2.csv'
img_dir = '../data/MoviePosters'

In [7]:
# Parameters?

useTransforms = False
genres = None

In [8]:
transformVar = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(p = 0.25),
    transforms.RandomVerticalFlip(p = 0.25),
    transforms.RandomRotation(60)
])

if not useTransforms:
    transformVar = None

In [9]:
pod = image_loader.PosterDataset(csv_file, img_dir, transform = transformVar, genres = genres)

In [1]:
# just a test to see if transformations work

# save_image(pod.__getitem__(0)[0] / 255, 'test.png')

In [5]:
from torch.utils.data import DataLoader

trainSize = int(len(pod) * 0.9)
testSize = len(pod) - trainSize

trainData, testData = torch.utils.data.random_split(pod, [trainSize, testSize])
def batch_function(raw_output):
    xs, ys = zip(*raw_output)
    batchxs = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)
    batchys = torch.Tensor(ys)
    return batchxs, batchys

BATCH_SIZE = 5
trainDataLoader = DataLoader(trainData, batch_size = BATCH_SIZE, shuffle=True, collate_fn=batch_function)
testDataLoader = DataLoader(testData, batch_size = 1, shuffle=True)

In [6]:
xs, ys = next(iter(trainDataLoader))
xs[0].shape

torch.Size([3, 268, 182])

In [ ]:
plt.imshow(xs[0].permute(1, 2, 0))

In [19]:
len(trainData)

31995

In [4]:
# for i in trainDataLoader:
#     print(i[0].dtype)

In [20]:
# download VGG19 model
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg19_bn(pretrained=True, progress=True).features.to(device).eval()

c:\Users\ianpe\.conda\envs\main\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ianpe\.conda\envs\main\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
# check model architecture
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [22]:
# Initialize the model
# num_features = model.features[-1].out_channels  # Get the number of output channels of the last convolutional layer
# model.features.add_module('7', nn.Conv2d(num_features, 1, 1))  # Add a 1x1 convolutional layer to change the output size to 1
model = model.to(device)

# Define the loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [23]:
class MoviePredictor(torch.nn.Module):
    def __init__(self, vggmodel, linear_params):
        super().__init__()
        self.vggmodel = vggmodel
        self.predictor_1 = torch.nn.Linear(512 * 8 * 8, linear_params)
        self.out = torch.nn.Linear(linear_params, 1)
    def forward(self, input: torch.Tensor):
        vgg_out = self.vggmodel(input).reshape(input.shape[0], 512 * 8 * 8)
        output = self.out(self.predictor_1(vgg_out))
        return output

In [30]:
predictor_model = MoviePredictor(model, linear_params= 512).to(device)

In [31]:
predictor_model(pod[1][0][None,:,:,:].to(device))

tensor([[-0.0465]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [32]:
predictor_model.vggmodel.requires_grad_(False)
predictor_optim = torch.optim.Adam(predictor_model.parameters(), lr=1e-4)

In [67]:
# Train the model
from tqdm import tqdm
import numpy as np

def train_model(in_model, criterion, optimizer, num_epochs):
    # uses global training data
    print(f"Training for {num_epochs} epoch(s).")
    in_model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        pbar = tqdm(trainDataLoader)
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = in_model(images)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            if np.isnan(loss.item()).any():
                print("NaN encountered in calculating loss, aborting")
                print(f"Predictions: {outputs}")
                print(f"Labels: {labels}")
                return

            running_loss += loss.item()
            pbar.set_description(f"Loss: {loss.item():.6f}")
        print(f'Epoch {epoch+1}, Average Loss: {running_loss/len(trainDataLoader):.6f}')

    # Test the model
    print("Testing Model")
    in_model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(testDataLoader):
            images, labels = images.to(device), labels.to(device)
            outputs = in_model(images)
            loss = criterion(outputs.squeeze(), labels.float().squeeze())
            test_loss += loss.item()
    print(f'Test Loss: {test_loss/len(testDataLoader)}')

In [34]:
train_model(predictor_model, nn.MSELoss(reduction="mean"), predictor_optim, 5)

Training for 5 epoch(s).


Loss: 1.9505285024642944: 100%|██████████| 6399/6399 [07:01<00:00, 15.17it/s]  


Epoch 1, Average Loss: 1.774296213962239


Loss: 0.8396715521812439: 100%|██████████| 6399/6399 [06:53<00:00, 15.47it/s]  


Epoch 2, Average Loss: 1.3357550720872706


...

Epoch 4, Average Loss: 1.146106440734609


Loss: 1.2600486278533936: 100%|██████████| 6399/6399 [06:39<00:00, 16.02it/s]  


Epoch 5, Average Loss: 1.102168035077679


100%|██████████| 3555/3555 [00:45<00:00, 77.59it/s]

Test Loss: 1.7783879402004412


In [35]:
# seems to be overfitting, but I'll do more training why not
predictor_optim = optim.Adam(predictor_model.parameters(), lr=1e-5)
loss_fn = nn.MSELoss(reduction="mean")
train_model(predictor_model, loss_fn, predictor_optim, 5)
torch.save(predictor_model.state_dict(), "../models/Predictor2Lin512.pt")

Training for 5 epoch(s).


Loss: 0.36939239501953125: 100%|██████████| 6399/6399 [07:16<00:00, 14.67it/s] 


Epoch 1, Average Loss: 0.9277733754208578


Loss: 1.756775140762329: 100%|██████████| 6399/6399 [07:24<00:00, 14.38it/s]   


Epoch 2, Average Loss: 0.9038602706226257


...

Epoch 4, Average Loss: 0.8818499675551221


Loss: 1.1599936485290527: 100%|██████████| 6399/6399 [07:13<00:00, 14.75it/s]   


Epoch 5, Average Loss: 0.876476656767355


100%|██████████| 3555/3555 [00:48<00:00, 73.64it/s]


Test Loss: 1.7219260570159998


In [41]:

torch.save(trainData, "../datasets/traindata.dataset")
torch.save(testData, "../datasets/testdata.dataset")

In [42]:
for x in tqdm(trainDataLoader):
    pass

100%|██████████| 6399/6399 [01:21<00:00, 78.95it/s]


In [51]:
class DeepDropPredictor(nn.Module):
    def __init__(self, vggmodel, dropout_p, num_extra_lin, hidden_dim):
        super().__init__()
        self.vgg_model = vggmodel
        self.flattened = nn.Linear(512 * 8 * 8, hidden_dim)
        self.lin_layers = nn.ParameterList([nn.Linear(hidden_dim, hidden_dim) for _ in range(num_extra_lin)])
        self.out = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(p=dropout_p)
    def forward(self, input: torch.Tensor):
        output = self.dropout(self.vgg_model(input).reshape(input.shape[0], 512 * 8 * 8))
        output = self.dropout(self.flattened(output))
        for layer in self.lin_layers:
            output = self.dropout(layer(output))
        output = self.out(output)
        return output

In [52]:
deepDropPredictor = DeepDropPredictor(model, 0.5, 2, 512).to(device)
deepDropPredictor.vgg_model.requires_grad_(False)
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-4)

train_model(deepDropPredictor, loss_fn, deepOptim, 3)

Training for 3 epoch(s).


Loss: 1.8596395254135132: 100%|██████████| 6399/6399 [06:49<00:00, 15.64it/s] 


Epoch 1, Average Loss: 2.9606533144912293


Loss: 3.578465223312378: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]   


Epoch 2, Average Loss: 2.2760654106849123


Loss: 0.6861000657081604: 100%|██████████| 6399/6399 [06:50<00:00, 15.59it/s]  


Epoch 3, Average Loss: 2.156359913838545


100%|██████████| 3555/3555 [00:48<00:00, 73.69it/s]

Test Loss: 2.220100910033067


In [53]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-5)

train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 3.026665210723877: 100%|██████████| 6399/6399 [06:50<00:00, 15.59it/s]   


Epoch 1, Average Loss: 1.753013373617619


Loss: 1.1039469242095947: 100%|██████████| 6399/6399 [06:51<00:00, 15.55it/s]  


Epoch 2, Average Loss: 1.6522094191074288


...

Epoch 4, Average Loss: 1.5690152994923936


Loss: 0.7266332507133484: 100%|██████████| 6399/6399 [07:04<00:00, 15.07it/s]  


Epoch 5, Average Loss: 1.5444395596618194


100%|██████████| 3555/3555 [00:51<00:00, 68.55it/s]


Test Loss: 1.8330523182734366


In [58]:
def test_model(in_model, criterion):
    in_model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(testDataLoader):
            images, labels = images.to(device), labels.to(device)
            outputs = in_model(images)
            loss = criterion(outputs.squeeze(), labels.float().squeeze())
            test_loss += loss.item()
    print(f'Test Loss: {test_loss/len(testDataLoader)}')

def test_empty_model(in_model, criterion):
    # in_model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(testDataLoader):
            images, labels = images.to(device), labels.to(device)
            outputs = in_model(images)
            loss = criterion(outputs.squeeze(), labels.float().squeeze())
            test_loss += loss.item()
    print(f'Test Loss: {test_loss/len(testDataLoader)}')

In [59]:
test_empty_model(lambda x: torch.Tensor([6.413]).to(device), loss_fn) # average score

100%|██████████| 3555/3555 [00:15<00:00, 222.46it/s]

Test Loss: 1.2836419191611301


In [60]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-6)
train_model(deepDropPredictor, loss_fn, deepOptim, 2)

Training for 2 epoch(s).


Loss: 1.1587564945220947: 100%|██████████| 6399/6399 [07:02<00:00, 15.16it/s]  


Epoch 1, Average Loss: 1.5048829187346664


Loss: 1.2700437307357788: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]  


Epoch 2, Average Loss: 1.4929980461582177


100%|██████████| 3555/3555 [00:47<00:00, 74.83it/s]

Test Loss: 1.4232098539967803


In [61]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-5)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 2.1003499031066895: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]  


Epoch 1, Average Loss: 1.5296387848145776


Loss: 1.0658739805221558: 100%|██████████| 6399/6399 [06:53<00:00, 15.46it/s]  


Epoch 2, Average Loss: 1.515018851160514


...

Epoch 4, Average Loss: 1.5061488871211293


Loss: 2.510054111480713: 100%|██████████| 6399/6399 [07:10<00:00, 14.87it/s]   


Epoch 5, Average Loss: 1.4934773749917405


100%|██████████| 3555/3555 [00:50<00:00, 70.23it/s]

Test Loss: 1.5648263567632892


In [62]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-4)
train_model(deepDropPredictor, loss_fn, deepOptim, 1)

Training for 1 epoch(s).


Loss: 0.34157657623291016: 100%|██████████| 6399/6399 [06:52<00:00, 15.52it/s] 


Epoch 1, Average Loss: 2.021788834831569


100%|██████████| 3555/3555 [00:47<00:00, 74.66it/s]

Test Loss: 1.9029515225770162


In [63]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-5)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 0.7813587188720703: 100%|██████████| 6399/6399 [06:48<00:00, 15.66it/s]  


Epoch 1, Average Loss: 1.6962225304342775


Loss: 1.3287068605422974: 100%|██████████| 6399/6399 [06:51<00:00, 15.56it/s]  


Epoch 2, Average Loss: 1.5597085176024992


...

Epoch 4, Average Loss: 1.4975741139133822


Loss: 1.428186058998108: 100%|██████████| 6399/6399 [06:51<00:00, 15.55it/s]   


Epoch 5, Average Loss: 1.4949296167531347


100%|██████████| 3555/3555 [00:47<00:00, 74.71it/s]

Test Loss: 1.604877375682443


In [65]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-6)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 4.137628555297852: 100%|██████████| 6399/6399 [06:46<00:00, 15.73it/s]   


Epoch 1, Average Loss: 1.455965


Loss: 1.1817193031311035: 100%|██████████| 6399/6399 [06:50<00:00, 15.59it/s]  


Epoch 2, Average Loss: 1.451000


...

Epoch 4, Average Loss: 1.440130


Loss: 1.4763487577438354: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]  


Epoch 5, Average Loss: 1.439623
Testing Model


100%|██████████| 3555/3555 [00:47<00:00, 74.62it/s]

Test Loss: 1.5110017234956894


In [68]:
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-7)
train_model(deepDropPredictor, loss_fn, deepOptim, 5)

Training for 5 epoch(s).


Loss: 2.065505: 100%|██████████| 6399/6399 [06:49<00:00, 15.64it/s]


Epoch 1, Average Loss: 1.426922


Loss: 1.551556: 100%|██████████| 6399/6399 [06:50<00:00, 15.58it/s]


Epoch 2, Average Loss: 1.420094


...

Epoch 4, Average Loss: 1.426952


Loss: 0.416120: 100%|██████████| 6399/6399 [06:46<00:00, 15.73it/s] 


Epoch 5, Average Loss: 1.415997
Testing Model


100%|██████████| 3555/3555 [00:46<00:00, 76.76it/s]

Test Loss: 1.4303539512436128


In [69]:
# just keep going why not
deepOptim = optim.Adam(deepDropPredictor.parameters(), lr=1e-7)
train_model(deepDropPredictor, loss_fn, deepOptim, 3)

Training for 3 epoch(s).


Loss: 0.686216: 100%|██████████| 6399/6399 [06:50<00:00, 15.60it/s]


Epoch 1, Average Loss: 1.420392


Loss: 1.540256: 100%|██████████| 6399/6399 [06:51<00:00, 15.54it/s]


Epoch 2, Average Loss: 1.425687


Loss: 1.138829: 100%|██████████| 6399/6399 [06:51<00:00, 15.57it/s]


Epoch 3, Average Loss: 1.427274
Testing Model


100%|██████████| 3555/3555 [00:48<00:00, 73.10it/s]

Test Loss: 1.7407562702686945


In [70]:
torch.save(deepDropPredictor.state_dict(), "../models/DeepDrop3-512.pt")

In [80]:
total_params = 0
for param in list(deepDropPredictor.parameters())[-8:]:
    print(param.size().numel())
    total_params += param.size().numel()
print(f"{total_params:,}")

16777216
512
262144
512
262144
512
512
1
17,303,553
